In [2]:
# import needed
import pandas as pd # data processing
import pandas_profiling as pp
import numpy as np # linear algebra
import pprint #indet json 
import requests #make http requests
from qwikidata.sparql  import return_sparql_query_results #return sparql results

In [5]:
df_netflix = pd.read_csv('originalDataset/netflix_titles.csv')
df_amazon = pd.read_csv('originalDataset/amazon_prime_titles.csv')
print(len(df_netflix))
print(len(df_amazon))

8807
9668


In [6]:
df_netflix.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,"September 25, 2021",2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,"September 24, 2021",2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...


In [7]:
df_amazon.head(3)

,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,The Grand Seduction,Don McKellar,"Brendan Gleeson, Taylor Kitsch, Gordon Pinsent",Canada,"March 30, 2021",2014,NaN,113 min,"Comedy, Drama",A small fishing village must procure a local d...
1,s2,Movie,Take Care Good Night,Girish Joshi,"Mahesh Manjrekar, Abhay Mahajan, Sachin Khedekar",India,"March 30, 2021",2018,13+,110 min,"Drama, International",A Metro Family decides to fight a Cyber Crimin...
2,s3,Movie,Secrets of Deception,Josh Webber,"Tom Sizemore, Lorenzo Lamas, Robert LaSardo, R...",United States,"March 30, 2021",2017,NaN,74 min,"Action, Drama, Suspense",After a man discovers his wife is cheating on ...


In [8]:
df_netflix["date_added"] = pd.to_datetime(df_netflix['date_added']) #turn dates into dates


""" df_netflix['month_added']=df_netflix['date_added'].dt.month
df_netflix['month_name_added']=df_netflix['date_added'].dt.month_name() #add separate column 4 year date month(?)
df_netflix['year_added'] = df_netflix['date_added'].dt.year
 """
df_netflix.head(3)


,show_id,type,title,director,cast,country,date_added,release_year,rating,duration,listed_in,description
0,s1,Movie,Dick Johnson Is Dead,Kirsten Johnson,NaN,United States,2021-09-25,2020,PG-13,90 min,Documentaries,"As her father nears the end of his life, filmm..."
1,s2,TV Show,Blood & Water,NaN,"Ama Qamata, Khosi Ngema, Gail Mabalane, Thaban...",South Africa,2021-09-24,2021,TV-MA,2 Seasons,"International TV Shows, TV Dramas, TV Mysteries","After crossing paths at a party, a Cape Town t..."
2,s3,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",NaN,2021-09-24,2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...


In [14]:
netflix = df_netflix.query("type == 'Movie'") #keep only movies
netflix.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
netflix.head(3)

NameError: name 'df_netflix' is not defined

In [13]:

#QUERY!!!!!!!!!!!!!

'''SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score
WHERE
{
  ?film wdt:P31 wd:Q11424 .
  ?film rdfs:label "Zubaan"@en .
  ?film rdfs:label ?film_label .
  FILTER(lang(?film_label) = 'en')
  ?film wdt:P57 ?director . 
  ?director rdfs:label ?director_label .    
  FILTER(lang(?director_label) = 'en')
  ?director wdt:P21 ?dir_gen . 
  ?dir_gen rdfs:label ?dir_gen_label .
  FILTER(lang(?dir_gen_label) = 'en')
  ?film wdt:P750 ?distributor . 
  ?distributor rdfs:label ?distributor_label .
  FILTER(lang(?distributor_label) = 'en')
  ?film wdt:P444 ?review_score .
}'''

'SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score\nWHERE\n{\n  ?film wdt:P31 wd:Q11424 .\n  ?film rdfs:label "Zubaan"@en .\n  ?film rdfs:label ?film_label .\n  FILTER(lang(?film_label) = \'en\')\n  ?film wdt:P57 ?director . \n  ?director rdfs:label ?director_label .    \n  FILTER(lang(?director_label) = \'en\')\n  ?director wdt:P21 ?dir_gen . \n  ?dir_gen rdfs:label ?dir_gen_label .\n  FILTER(lang(?dir_gen_label) = \'en\')\n  ?film wdt:P750 ?distributor . \n  ?distributor rdfs:label ?distributor_label .\n  FILTER(lang(?distributor_label) = \'en\')\n  ?film wdt:P444 ?review_score .\n}'

In [14]:
p = pprint.PrettyPrinter(indent=1)
# funzia piu o mno come una ricerca su wikidata
API_WD = "https://www.wikidata.org/w/api.php"
params = {
    'action': 'wbsearchentities',# <------------ web search from wikidata
    'format': 'json',
    'language': 'en',
    'search': 'Dick Johnson Is Dead' # the query string
}

# query wd API    
r = requests.get(API_WD, params = params).json() 
p.pprint(r)

{'search': [{'concepturi': 'http://www.wikidata.org/entity/Q84366622',
             'description': '2020 documentary film by Kirsten Johnson',
             'display': {'description': {'language': 'en',
                                         'value': '2020 documentary film by '
                                                  'Kirsten Johnson'},
                         'label': {'language': 'en',
                                   'value': 'Dick Johnson Is Dead'}},
             'id': 'Q84366622',
             'label': 'Dick Johnson Is Dead',
             'match': {'language': 'en',
                       'text': 'Dick Johnson Is Dead',
                       'type': 'label'},
             'pageid': 83667587,
             'repository': 'wikidata',
             'title': 'Q84366622',
             'url': '//www.wikidata.org/wiki/Q84366622'}],
 'searchinfo': {'search': 'Dick Johnson Is Dead'},
 'success': 1}


In [15]:
from SPARQLWrapper import SPARQLWrapper, JSON #questo serve a vedere la struttura delle risposte
import ssl

p = pprint.PrettyPrinter(indent=1)
ssl._create_default_https_context = ssl._create_unverified_context #????? ceracare poi (lezioni daquino)

# get the endpoint API
wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql" #<---------- Non è l'endpoint, stiamo fetchando della documentazione di qualche tipo

# prepare the query : 10 random triples
my_SPARQL_query = """
SELECT *
WHERE {?s ?p ?o}
LIMIT 10
"""

# set the endpoint 
sparql_wd = SPARQLWrapper(wikidata_endpoint)
# set the query
sparql_wd.setQuery(my_SPARQL_query)
# set the returned format
sparql_wd.setReturnFormat(JSON)
# get the results
results = sparql_wd.query().convert()

p.pprint(results)

{'head': {'vars': ['s', 'p', 'o']},
 'results': {'bindings': [{'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
                                 'type': 'literal',
                                 'value': '2021-09-24T23:11:29Z'},
                           'p': {'type': 'uri',
                                 'value': 'http://schema.org/dateModified'},
                           's': {'type': 'uri',
                                 'value': 'http://wikiba.se/ontology#Dump'}},
                          {'o': {'datatype': 'http://www.w3.org/2001/XMLSchema#dateTime',
                                 'type': 'literal',
                                 'value': '2021-09-24T23:11:31Z'},
                           'p': {'type': 'uri',
                                 'value': 'http://schema.org/dateModified'},
                           's': {'type': 'uri',
                                 'value': 'http://wikiba.se/ontology#Dump'}},
                          {'o': {'datatype':

In [3]:
import time
for x in range(6):
  if x == 3:
    time.sleep(5)
    print(x)
  else:
    print(x)


0
1
2
3
4
5


In [17]:
title = list(netflix['title']) #printa una lista di titoli da netflix
#print(title[0:100], title[100:200])
#print(title[0:100], title[100:200])
print(len(title))


6131


In [22]:
#print(film)
ciccio_prova = title[0:20] 
print(ciccio_prova)


['Dick Johnson Is Dead', 'My Little Pony: A New Generation', 'Sankofa', 'The Starling', 'Je Suis Karl', 'Confessions of an Invisible Girl', "Europe's Most Dangerous Man: Otto Skorzeny in Spain", 'Intrusion', 'Avvai Shanmughi', 'Go! Go! Cory Carson: Chrissy Takes the Wheel', 'Jeans', 'Minsara Kanavu', 'Grown Ups', 'Dark Skies', 'Paranoia', 'Ankahi Kahaniya', 'The Father Who Moves Mountains', 'The Stronghold', 'Birth of the Dragon', 'Jaws']


In [ ]:
""" print("film1 should be []")
print(film) """


p = pprint.PrettyPrinter(indent=1)

def wikidata_reconciliation(query):

    oneundred_ttls = [ciccio_prova[x:x + 100] for x in range(0, len(ciccio_prova), 100)]
    for i in range(0, len(oneundred_ttls)):
        time.sleep(25)
        

        ssl._create_default_https_context = ssl._create_unverified_context
    
    
    # get the endpoint API
        wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        """ print("query = title[0:2]")
        print(query)
        print("film should be []")
        print(film) """

        for x in oneundred_ttls[i]:
            print(x)
            my_SPARQL_query = """
            SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            ?film wdt:P57 ?director . 
            ?director rdfs:label ?director_label .    
            FILTER(lang(?director_label) = 'en')
            ?director wdt:P21 ?dir_gen . 
            ?dir_gen rdfs:label ?dir_gen_label .
            FILTER(lang(?dir_gen_label) = 'en')
            ?film wdt:P750 ?distributor . 
            ?distributor rdfs:label ?distributor_label .
            FILTER(lang(?distributor_label) = 'en')
            ?film wdt:P444 ?review_score .
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()
            response = requests.head(wikidata_endpoint)
            """if response.status_code == 429:
                print("riproviamo")
                time.sleep(int(response.headers["Retry-After"]))"""
            
            if results['results']['bindings'] == []:
                print(""+x+" not found")
            else:
                film.append(results['results']['bindings'][0]['film_label']['value'])
                director.append(results['results']['bindings'][0]['director_label']['value'])
                gender.append(results['results']['bindings'][0]['dir_gen_label']['value'])
                distributor.append(results['results']['bindings'][0]['distributor_label']['value'])
                    
            
    print(len(film))



wikidata_reconciliation(ciccio_prova)

#wikidata_reconciliation(title[100:200])


Dick Johnson Is Dead
My Little Pony: A New Generation
My Little Pony: A New Generation not found
Sankofa
Sankofa not found
The Starling
Je Suis Karl
Je Suis Karl not found
Confessions of an Invisible Girl
Confessions of an Invisible Girl not found
Europe's Most Dangerous Man: Otto Skorzeny in Spain
Europe's Most Dangerous Man: Otto Skorzeny in Spain not found
Intrusion
Intrusion not found
Avvai Shanmughi
Avvai Shanmughi not found
Go! Go! Cory Carson: Chrissy Takes the Wheel
Go! Go! Cory Carson: Chrissy Takes the Wheel not found
Jeans
Jeans not found
Minsara Kanavu
Minsara Kanavu not found
Grown Ups
Dark Skies
Paranoia
Ankahi Kahaniya
Ankahi Kahaniya not found
The Father Who Moves Mountains
The Father Who Moves Mountains not found
The Stronghold
The Stronghold not found
Birth of the Dragon
Jaws
Jaws 2
Jaws 3
Jaws 3 not found
Jaws: The Revenge
My Heroes Were Cowboys
My Heroes Were Cowboys not found
Safe House
Training Day
InuYasha the Movie 2: The Castle Beyond the Looking Glass
InuYasha

HTTPError: HTTP Error 429: Too Many Requests

# New

In [70]:
df_newdata = pd.read_csv('new_data.csv')
df_newdata = df_newdata.loc[:, ~df_newdata.columns.str.contains('^Unnamed')]
df_newdata.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
df_newdata['title'] = df_newdata['title'].replace({'"':''}, regex=True)

df_newdata.head(2)
#print(len(df_newdata))

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,TV Show,Ganglands,Julien Leclercq,"Sami Bouajila, Tracy Gotoas, Samuel Jouy, Nabi...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"Crime TV Shows, International TV Shows, TV Act...",To protect his family from a powerful drug lor...,1,0,1000
1,TV Show,Midnight Mass,Mike Flanagan,"Kate Siegel, Zach Gilford, Hamish Linklater, H...",No Data,"September 24, 2021",2021,TV-MA,1 Season,"TV Dramas, TV Horror, TV Mysteries",The arrival of a charismatic young priest brin...,1,0,1000


In [71]:
movie_to_find = df_newdata.query("type == 'Movie'") #keep only movies
serie_to_find = df_newdata.query("type == 'TV Show'")


movie_to_find.reset_index(level=None, drop=True, inplace=True, col_level=0, col_fill='')
movie_to_find.head(1)

,type,title,director,cast,country,date_added_netflix,release_year,rating,duration,listed_in,description,netflix,amazon,date_added_amazon
0,Movie,My Little Pony: A New Generation,"Robert Cullen, José Luis Ucha","Vanessa Hudgens, Kimiko Glenn, James Marsden, ...",No Data,"September 24, 2021",2021,PG,91 min,Children & Family Movies,Equestria's divided. But a bright-eyed hero be...,1,0,1000


In [72]:
print(len(df_newdata))
print(len(movie_to_find))
print(len(serie_to_find))
print(12214 -12036)

12214
12036
178
178


In [76]:
movie_title = list(movie_to_find['title'])

print(movie_title[0:60])
print(movie_title.index("Gabriel Fluffy Iglesias: One Show Fits All"))
#print(movie_title[2399:])


['My Little Pony: A New Generation', 'Sankofa', 'The Starling', 'Je Suis Karl', 'Confessions of an Invisible Girl', 'Intrusion', 'Avvai Shanmughi', 'Go! Go! Cory Carson: Chrissy Takes the Wheel', 'Jeans', 'Minsara Kanavu', 'Grown Ups', 'Dark Skies', 'Paranoia', 'Ankahi Kahaniya', 'The Father Who Moves Mountains', 'The Stronghold', 'Birth of the Dragon', 'Jaws', 'Jaws 2', 'Jaws 3', 'Jaws: The Revenge', 'Safe House', 'Training Day', 'InuYasha the Movie 2: The Castle Beyond the Looking Glass', 'InuYasha the Movie 3: Swords of an Honorable Ruler', 'InuYasha the Movie 4: Fire on the Mystic Island', 'InuYasha the Movie: Affections Touching Across Time', 'Naruto Shippuden the Movie: Blood Prison', 'Naruto Shippûden the Movie: Bonds', 'Naruto Shippûden the Movie: The Will of Fire', 'Naruto Shippuden: The Movie', 'Naruto Shippuden: The Movie: The Lost Tower', 'Naruto the Movie 2: Legend of the Stone of Gelel', 'Naruto the Movie 3: Guardians of the Crescent Moon Kingdom', 'Naruto the Movie: Ninj

In [ ]:
# old query
"""
            SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            OPTIONAL {?film wdt:P57 ?director . 
            ?director rdfs:label ?director_label .    
            FILTER(lang(?director_label) = 'en')}
            OPTIONAL {?director wdt:P21 ?dir_gen . 
            ?dir_gen rdfs:label ?dir_gen_label .
            FILTER(lang(?dir_gen_label) = 'en')}
             OPTIONAL {?film wdt:P750 ?distributor . 
            ?distributor rdfs:label ?distributor_label .
            FILTER(lang(?distributor_label) = 'en')}
            }
"""

In [19]:
from SPARQLWrapper import SPARQLWrapper, JSON #questo serve a vedere la struttura delle risposte
import ssl

p = pprint.PrettyPrinter(indent=1)

In [ ]:
# "In Your Hands" == error 500

In [ ]:
distributor = []
imdbID = []
rottenscore = []


In [ ]:
film = [] #facciamo un po' di liste che ci divertiamo
director = []
gender = []


In [62]:




p = pprint.PrettyPrinter(indent=1)
import urllib.error
from xml.etree.ElementPath import xpath_tokenizer_re

def wikidata_reconciliation(query):

    
    # get the endpoint API
    wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        

    for x in query:

        try:

            
            my_SPARQL_query = """
            SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label 
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            OPTIONAL {?film wdt:P57 ?director . 
            ?director rdfs:label ?director_label .    
            FILTER(lang(?director_label) = 'en')}
            OPTIONAL {?director wdt:P21 ?dir_gen . 
            ?dir_gen rdfs:label ?dir_gen_label .
            FILTER(lang(?dir_gen_label) = 'en')}
            
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()

            if results['results']['bindings'] == []:
                print(""+x+" not found")
            else:
                film.append(results['results']['bindings'][0]['film_label']['value'])
                if "director_label" in results['results']['bindings'][0]:
                    director.append(results['results']['bindings'][0]['director_label']['value'])
                else:
                    director.append("no_data")
                if "dir_gen_label" in results['results']['bindings'][0]:
                    gender.append(results['results']['bindings'][0]['dir_gen_label']['value'])
                else:
                    gender.append("no_data")
                


        except urllib.error.HTTPError as e:

            print(e.headers["retry-after"])
            time.sleep((int(e.headers["retry-after"])) + 1)
            error_title = query.index(x)
            wikidata_reconciliation(query[error_title:])
            
   



wikidata_reconciliation(movie_title)

My Little Pony: A New Generation not found
Je Suis Karl not found


JSONDecodeError: Expecting value: line 2223971 column 19 (char 58466301)

In [58]:
#print(not_found)

['My Little Pony: A New Generation', 'Je Suis Karl']


In [63]:

p = pprint.PrettyPrinter(indent=1)
import urllib.error
from xml.etree.ElementPath import xpath_tokenizer_re

def wikidata_reconciliation1(query):

    
    # get the endpoint API
    wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
        

    for x in query:

        try:

            
            my_SPARQL_query = """
            SELECT ?film ?film_label ?distributor ?distributor_label ?imdbID ?rottenscore
            WHERE
            {
            ?film wdt:P31 wd:Q11424 .
            ?film rdfs:label """+'"'+ x +'"' +"""@en .
            ?film rdfs:label ?film_label .
            FILTER(lang(?film_label) = 'en')
            OPTIONAL {?film wdt:P750 ?distributor . 
            ?distributor rdfs:label ?distributor_label .
            FILTER(lang(?distributor_label) = 'en')}
            OPTIONAL {?film wdt:P345 ?imdbID.}
            OPTIONAL {?film wdt:P444 ?rottenscore.}
            }
            """
            # set the endpoint 
            sparql_wd = SPARQLWrapper(wikidata_endpoint)
            # set the query
            sparql_wd.setQuery(my_SPARQL_query)
            # set the returned format
            sparql_wd.setReturnFormat(JSON)
            # get the results
            
            results = sparql_wd.query().convert()

            if results['results']['bindings'] == []:
                print(""+x+" not found")
            else:
                if "distributor_label" in results['results']['bindings'][0]:
                    distributor.append(results['results']['bindings'][0]['distributor_label']['value'])
                else:
                    distributor.append("no_data")
                if "imdbID" in results['results']['bindings'][0]:
                    imdbID.append(results['results']['bindings'][0]['imdbID']['value'])
                else:
                    imdbID.append("no_data")
                if "rottenscore" in results['results']['bindings'][0]:
                    rottenscore.append(results['results']['bindings'][0]['rottenscore']['value'])
                else:
                    rottenscore.append("no_data")



        except urllib.error.HTTPError as e:

            print(e.headers["retry-after"])
            time.sleep((int(e.headers["retry-after"])) + 1)
            error_title = query.index(x)
            wikidata_reconciliation1(query[error_title:])
            




wikidata_reconciliation1(movie_title)

My Little Pony: A New Generation not found
Je Suis Karl not found
Go! Go! Cory Carson: Chrissy Takes the Wheel not found
Jaws 3 not found
InuYasha the Movie 2: The Castle Beyond the Looking Glass not found
InuYasha the Movie 3: Swords of an Honorable Ruler not found
InuYasha the Movie 4: Fire on the Mystic Island not found
InuYasha the Movie: Affections Touching Across Time not found
Naruto Shippuden the Movie: Blood Prison not found
Naruto Shippûden the Movie: Bonds not found
Naruto Shippûden the Movie: The Will of Fire not found
Naruto Shippuden: The Movie not found
Naruto Shippuden: The Movie: The Lost Tower not found
Naruto the Movie 2: Legend of the Stone of Gelel not found
Naruto the Movie 3: Guardians of the Crescent Moon Kingdom not found
Naruto the Movie: Ninja Clash in the Land of Snow not found
A StoryBots Space Adventure not found
You vs. Wild: Out Cold not found
Little Singham - Black Shadow not found
Tughlaq Durbar (Telugu) not found
Firedrake the Silver Dragon not found


QueryBadFormed: QueryBadFormed: a bad request has been sent to the endpoint, probably the sparql query is bad formed. 

Response:
b'SPARQL-QUERY: queryStr=\n            SELECT ?film ?film_label ?distributor ?distributor_label ?imdbID ?rottenscore\n            WHERE\n            {\n            ?film wdt:P31 wd:Q11424 .\n            ?film rdfs:label "Gabriel "Fluffy" Iglesias: One Show Fits All"@en .\n            ?film rdfs:label ?film_label .\n            FILTER(lang(?film_label) = \'en\')\n            OPTIONAL {?film wdt:P750 ?distributor . \n            ?distributor rdfs:label ?distributor_label .\n            FILTER(lang(?distributor_label) = \'en\')}\n            OPTIONAL {?film wdt:P345 ?imdbID.}\n            OPTIONAL {?film wdt:P444 ?rottenscore.}\n            }\n            \njava.util.concurrent.ExecutionException: org.openrdf.query.MalformedQueryException: Lexical error at line 6, column 46.  Encountered: "\\"" (34), after : "Fluffy"\n\tat java.util.concurrent.FutureTask.report

In [65]:
print(film)
print(director)
print(gender)
print(len(distributor))
print(len(imdbID))
print(len(rottenscore))

['Sankofa', 'The Starling']
['Haile Gerima', 'Theodore Melfi']
['male', 'male']
1570
1570
1570


In [ ]:
p = pprint.PrettyPrinter(indent=1)

def wikidata_reconciliation(query):

    ssl._create_default_https_context = ssl._create_unverified_context
    
    # get the endpoint API
    wikidata_endpoint = "https://query.wikidata.org/bigdata/namespace/wdq/sparql"
    """ print("query = title[0:2]")
    print(query)
    print("film should be []")
    print(film) """

    for x in query:
        print(x)
        my_SPARQL_query = """
        SELECT ?film ?film_label ?director ?director_label ?dir_gen ?dir_gen_label ?distributor ?distributor_label ?review_score
        WHERE
        {
        ?film wdt:P31 wd:Q11424 .
        ?film rdfs:label """+'"'+ x +'"' +"""@en .
        ?film rdfs:label ?film_label .
        FILTER(lang(?film_label) = 'en')
        ?film wdt:P57 ?director . 
        ?director rdfs:label ?director_label .    
        FILTER(lang(?director_label) = 'en')
        ?director wdt:P21 ?dir_gen . 
        ?dir_gen rdfs:label ?dir_gen_label .
        FILTER(lang(?dir_gen_label) = 'en')
        ?film wdt:P750 ?distributor . 
        ?distributor rdfs:label ?distributor_label .
        FILTER(lang(?distributor_label) = 'en')
        ?film wdt:P444 ?review_score .
        }
        """
        # set the endpoint 
        sparql_wd = SPARQLWrapper(wikidata_endpoint)
        # set the query
        sparql_wd.setQuery(my_SPARQL_query)
        # set the returned format
        sparql_wd.setReturnFormat(JSON)
        # get the results
        results = sparql_wd.query().convert()
        # get response
        response = requests.head(wikidata_endpoint)
        print(response.status_code)
        if response.status_code != 200:
            p.pprint(response.headers)

    print(response.status_code)
    if response.status_code != 200:
            p.pprint(response.headers)
        
wikidata_reconciliation(title[0:140])
        


Dick Johnson Is Dead
200
{'server': 'nginx/1.14.2', 'date': 'Sat, 13 Aug 2022 09:33:54 GMT', 'content-type': 'application/rdf+xml', 'x-served-by': 'wdqs1007', 'access-control-allow-origin': '*', 'cache-control': 'public, max-age=300', 'content-encoding': 'gzip', 'vary': 'Accept, Accept-Encoding', 'age': '16', 'x-cache': 'cp6013 miss, cp6013 pass', 'x-cache-status': 'pass', 'server-timing': 'cache;desc="pass", host;desc="cp6013"', 'strict-transport-security': 'max-age=106384710; includeSubDomains; preload', 'report-to': '{ "group": "wm_nel", "max_age": 86400, "endpoints": [{ "url": "https://intake-logging.wikimedia.org/v1/events?stream=w3c.reportingapi.network_error&schema_uri=/w3c/reportingapi/network_error/1.0.0" }] }', 'nel': '{ "report_to": "wm_nel", "max_age": 86400, "failure_fraction": 0.05, "success_fraction": 0.0}', 'set-cookie': 'WMF-Last-Access=13-Aug-2022;Path=/;HttpOnly;secure;Expires=Wed, 14 Sep 2022 00:00:00 GMT, WMF-Last-Access-Global=13-Aug-2022;Path=/;Domain=.wikidata.o

In [47]:
print(film)

[]


In [24]:
print(film)


['Dick Johnson Is Dead', 'The Starling', 'Grown Ups', 'Dark Skies', 'Paranoia', 'Birth of the Dragon', 'Jaws']
